# **Question 1**

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
#Installing required packages
!pip install contractions

In [34]:
#Installing necessary libraries
import os
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
import re
import string
import contractions
import json
from textblob import TextBlob, Word
import joblib
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lema = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
#Loading the dataset
file_list = os.listdir(r"/content/drive/MyDrive/Datasets/Humor,Hist,Media,Food")
print(file_list)

['gohome.hum', 'rinaldos.txt', 'mowers.txt', 'harmful.hum', 'hell.jok', 'mothers.txt', 'herb!.hum', 'hate.hum', 'hi.tec', 'roach.asc', 'icm.hum', 'conan.txt', 'howlong.hum', 'murph.jok', 'imprrisk.hum', 'coyote.txt', 'horoscop.jok', 'murphy.txt', 'impurmat.hum', 'incarhel.hum', 'bible.txt', 'dirtword.txt', 'insanity.hum', 'insure.hum', 'cartoon_.txt', 'investi.hum', 'interv.hum', 'bless.bc', 'jac&tuu.hum', 'japrap.hum', 'terms.hum', 'ivan.hum', 'test.jok', 'dym', 'killer.hum', 'blooprs1.asc', 'kilsmur.hum', 'testchri.txt', 'killself.hum', 'lawsuniv.hum', 'test2.jok', 'kid_diet.txt', 'motrbike.jok', 'lbinter.hum', 'murphys.txt', 'ludeinfo.txt', 'legal.hum', 'simp.txt', 'office.txt', 'llong.hum', 'lll.hum', 'livnware.hum', 'lobquad.hum', 'lifeinfo.hum', 'lifeimag.hum', 'lif&love.hum', 'looser.hum', 'phunatdi.ana', 'losers84.hum', 'ludeinfo.hum', 'lozerzon.hum', 'lozeuser.hum', 'catstory.txt', 'losers86.hum', 'luggage.hum', 'm0dzmen.hum', 'makebeer.hum', 'manilla.hum', 'manspace.hum', 'ma

In [36]:
#Check the number of files present
len(file_list)

1135

## **(a) Preprocessing Steps**

In [37]:
doc = {}
lp = len(string.punctuation)
#Cleaning the data
def cleaning(flag, path, name):
  #For pre-processing queries
  if(flag==0):
    #Expand contractions
    #path = contractions.fix(path)
    #Convert to lower case
    path = path.lower()
    #Remove punctuations and replace with space
    path = path.translate(str.maketrans(string.punctuation, " "*lp, ""))
    path = re.sub('[^A-Za-z\s\n ]+', ' ',path)
    #Function for tokenization
    path = word_tokenize(path)
    #Function for stop words removal and lemmatization
    path = [lema.lemmatize(s) for s in path if s not in stopwords.words('english') and s.isalpha and len(s)>1]
    return path

  else:
    #For pre-processing of files
    rf = open(path,'r',errors = 'ignore')
    #Read the file
    rf = rf.read()
    #Convert to lower case
    clean = rf.lower()
    #Remove punctuations and replace with space
    clean = clean.translate(str.maketrans(string.punctuation, " "*lp, ""))
    clean = re.sub('[^A-Za-z\s\n ]+', ' ',clean)
    #Function for tokenization
    clean = word_tokenize(clean)
    #Function for stop words removal and lemmatization
    clean = [lema.lemmatize(s) for s in clean if s not in stopwords.words('english') and s.isalpha and len(s)>1]
    doc[name] = clean

In [38]:
for file in file_list:
  cleaning(1,"/content/drive/MyDrive/Datasets/Humor,Hist,Media,Food/"+file,file)
  #flag = 1 for pre-processing of files

In [39]:
len(doc)

1135

## **(b) Unigram Inverted Index Data Structure**

In [40]:
invertedIndex = {}
final = []
#Final set of words
file = open('vocabulary.txt','w')
for doc_name in doc:
    for word in doc[doc_name]:
        final.append(word)
final = set(final)
for word in final:
   file.write(word+" , ")
file.close
#invertedIndex (dictinary) contains term and posting list
#Build posting list containing all doc id's where term exist
post = open('posting.txt','w') 
for term in final:
    postingList = []
    post.write("\n"+term + " - ")
    for id in doc:
        if term in doc[id]:
            postingList.append(id)
            post.write(id+" , ")
    invertedIndex[term] = postingList 

In [41]:
#dump the invertedIndex using joblib
joblib.dump(invertedIndex, '/content/drive/MyDrive/invertedIndex')

['/content/drive/MyDrive/invertedIndex']

In [42]:
#Load invertedIndex using joblib in postings
invertedIndex = joblib.load('/content/drive/MyDrive/invertedIndex')
postings = invertedIndex

## **(c) Query Processing**

In [43]:
#Pre-process queries and operators
#Use flag = 0
qp=[]
def qprocess(operator, query):
  #Pre-process the given operator 
  operator = re.sub("[^a-zA-Z\s\n\,]", " ",operator)
  op = operator.split(",")
  op = [x.strip(" ") for x in op]
  op = [x.upper() for x in op]
  #Pre-process the given query 
  qp = cleaning(0, query, " ") 
  return op, qp 

## **(i)   x OR y**
## **(ii)   x AND y** 
## **(iii)   x AND NOT y**
## **(iv)   x OR NOT y** 



In [44]:
def merge_pl(op, qfinal):
  c = 0
  w1 = qfinal[0]
  w2 = qfinal[1]
  doc1=[]
  doc2=[]
  finalList=[]


  for i in range(len(qfinal)-1):
      #Take two words at a time, find posting list, add to the final doc and remove intersection
      #Finding concatenation
      if op[i] == 'OR':
        if i == 0 :
          if (w1 in postings.keys()):
           doc1 = postings[w1].copy()
          if (w2 in postings.keys()):
           doc2 = postings[w2].copy()
        else :
          doc1=[]
          doc1 = finalList
          finalList = []
          w2 = qfinal[i+1]
          if (w2 in postings.keys()):
              doc2=[]
              doc2 = postings[w2].copy()
        for item in doc1:
            c += 1
            finalList.append(item)
            if item in doc2:
              doc2.remove(item)
        for item in doc2:
          c += 1
          finalList.append(item)

      #Take two words at a time, find posting list, find intersection and add to the final doc 
      if op[i] == 'AND':
        if i == 0 :
          if (w1 in postings.keys()):
           doc1 = postings[w1].copy()
          if (w2 in postings.keys()):
           doc2 = postings[w2].copy()
        else :
          doc1=[]
          doc1 = finalList
          finalList = []
          w2 = qfinal[i+1]
          if (w2 in postings.keys()):
              doc2 = []
              doc2 = postings[w2].copy()
        for item in doc1:
            if item in doc2:
              c +=1
              finalList.append(item)  

      #Find posting list of two words, add posting list of w1 in final doc, remove intersection of w1 and w2 posting list          
      if op[i] == 'AND NOT':
        if i == 0 :
          if (w1 in postings.keys()):
           doc1 = postings[w1].copy()
          if (w2 in postings.keys()):
           doc2 = postings[w2].copy()
          for item in doc1:
            c += 1
            finalList.append(item)
            if item in doc2:
              finalList.remove(item)           
        else :
          doc1=[]
          doc1 = finalList
          finalList = []
          w2 = qfinal[i+1]
          if (w2 in postings.keys()):
              doc2=[]
              doc2 = postings[w2].copy()
          for item in doc1:
            finalList.append(item)
            if item in doc2:
              finalList.remove(item)
              c += 1

      #Find posting list of w1 and find postings in which 2 is not present, then do OR operation
      if op[i] == 'OR NOT':
        if i == 0 :
          if (w1 in postings.keys()):
           doc1 = postings[w1].copy()
          for item in postings.keys():
             if item != w2:
               for u in postings[item]:
                  if u not in doc2:
                    doc2.append(u)       
        else :
          doc1 = []
          doc1 = finalList
          finalList = []

          w2 = qfinal[i+1]
          doc2 = []
          for item in postings.keys():
             if item != w2:
               for u in postings[item]:
                    if u not in doc2:
                       doc2.append(u)
        for item in doc1:
            c += 1
            finalList.append(item)
            if item in doc2:
              doc2.remove(item)
           
        for item in doc2:
          c += 1
          finalList.append(item)


  print("Number of documents matched: ", len(finalList))
  print("No. of comparisons required: ", c)
  print(finalList)

## **(d) Evaluation of queries**

In [45]:
print("Number of queries: ")
N = int(input())
for j in range(N):
  print("Input query: ")
  query = input()
  print("Input operation sequence: ")
  operation = input()
  op, qfinal = qprocess(operation, query)
  merge_pl(op, qfinal)

Number of queries: 
2
Input query: 
lion stood thoughtfully for a moment
Input operation sequence: 
OR, OR , OR
Number of documents matched:  213
No. of comparisons required:  414
['murphys.txt', 'llong.hum', 'onetotwo.hum', 'deep.txt', 'drunk.txt', 'hecomes.jok', 'wagon.hum', 'lion.jok', 'puzzles.jok', 'lions.cat', 'tnd.1', 'boneles2.txt', 'dthought.txt', 'tpquotes.txt', 'japantv.txt', 'mindvox', 'lion.txt', 'booze1.fun', 'stuf10.txt', 'gd_gal.txt', 'christop.int', 'filmgoof.txt', 'dromes.txt', 'three.txt', 'bitnet.txt', 'collected_quotes.txt', 'computer.txt', 'murphy_l.txt', 'lposting.json', 'posting_doc.json', 'conan.txt', 'ivan.hum', 'lifeimag.hum', 'lozerzon.hum', 'luggage.hum', 'm0dzmen.hum', 'maecenas.hum', 'reasons.txt', 'montpyth.hum', 'news.hum', 'onetoone.hum', 'peatchp.hum', 'pizzawho.hum', 'phorse.hum', 'passage.hum', 'quest.hum', 'fuckyou2.txt', 'smurfkil.hum', 'soleleer.hum', 'stone.hum', 'lawyer.jok', 'marriage.hum', 'misc.1', 'xibovac.txt', 'childhoo.jok', 'nigel.5', '

# **Question 2**

In [46]:
#Load the dataset
path = "/content/drive/MyDrive/Datasets/Humor,Hist,Media,Food"
os.chdir(path)

## **(a) Pre-processing**

In [47]:
ln = len(string.punctuation)
#Cleaning the data
def process(content):
  #Convert the text to lower case
  content = content.lower()
  #Remove punctuation marks from tokens
  content = content.translate(str.maketrans(string.punctuation, " "*ln,''))
  #Perform word tokenization
  ctokens = word_tokenize(content)
  #Remove stopwords from tokens and do lemmatization
  #Checking length, if length = 1
  ctokens = [lema.lemmatize(s) for s in ctokens if s not in stopwords.words('english') and s.isalpha and len(s)>1]
  return ctokens

## **(b) Positional index data structure**

In [48]:
#Function to create posting list
#Positional index
pol={}
def lposting(c, tl):
  i = 0
  for t in tl:
    i = i+1
    if t in pol:
      d = pol[t][1]
      if c in d:
        d[c].append(i)
      else:
        pol[t][0] = (pol[t][0]+1)
        d[c] = [i]
      pol[t][1] = d
    else:
      pol[t] = [] 
      pol[t].append(1)
      pol[t].append({})
      pol[t][1][c] = [i]

In [49]:
#Iterate over the dataset for preprocessing and indexing
c = 0
for file in glob.glob("*"):
  c = c+1
  #Reading the file
  with open(file,'r',encoding = 'utf8', errors = 'ignore') as f:
    rfile = f.read()
    doc[c] = file
    tl = process(rfile)
    lposting(c,tl)

print(c)    

1137


In [ ]:
#Print terms in posting list
for t in pol:
  print(t,pol[t][0],len(pol[t][1]))

In [51]:
#Dumping the posting index to avoid making posting index from scratch every time
save = open('lposting.json','w')
save = json.dump(pol,save)

In [52]:
#Loading the posting index
load_file = open('posting_doc.json','w')
load_file = json.dump(pol,load_file)

## **(c) Support for the searching of phrase queries**

In [53]:
#Checking common documents for the query in the list
def dcommon(qf):
  #Store common documents
  common = []
  n = len(qf)
  l = pol[qf[0]][1]
  for d in l:
    c=1
    for i in range(1, n):
      if d in pol[qf[i]][1]:
        c = c+1
    if c == n:
      common.append(d)
  return common  

In [54]:
#Retrieving the list of positions for words from the common documents
def word_pos(qf,cd):
  pol1 = {}
  for d in cd:
    for t in qf:
      if d in pol1:
        pol1[d].append(pol[t][1][d])
      else:
        pol1[d] = [pol[t][1][d]]

  return pol1    

## **(d) Query processing and evaluation**

In [55]:
#Function for query processing
def qprocess(qf):
  f_doc = []
  print(qf) 
  common = dcommon(qf)
  plist1 = word_pos(qf,common)
  for d in common:
    f = False
    tp = plist1[d]
    tp1 = tp[0]
    for i in range(len(tp1)):
      f1 = True
      c = tp1[i]
      for j in range(1,len(tp)):
        c = c+1
        if c not in tp[j]:
          f1 = False
          break
      if(f1):
        f = True
    if(f):
      f_doc.append(d)
    f_doc = list(set(f_doc))  
  return f_doc

In [56]:
#Evaluate query
print("Enter Query")
#Input the query
query = input()
#preprocessing input query
qf = process(query)
#Processing input query
d = qprocess(qf)
#Listing documents
final_doc = []
for i in d:
  final_doc.append(doc[i])

print(f"\nThe number of documents retrieved : {len(d)}")
print("The list of document names retrieved : ")
print(final_doc)

Enter Query
Good Day
['good', 'day']

The number of documents retrieved : 9
The list of document names retrieved : 
['xibovac.txt', 'tnd.1', 'onetoone.hum', 'horoscop.jok', 'horoscop.txt', 'lozeuser.hum', 'humor9.txt', "terrmcd'.hum", 'valujet.txt']
